In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import findspark
findspark.init()

import pyspark
from pyspark.sql import *
import pyspark.sql.functions as func
from pyspark.sql.types import *

%run insights.py
%run plot.py

In [3]:
spark = SparkSession.builder.getOrCreate()

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: org.apache.spark.SparkException: Only one SparkContext may be running in this JVM (see SPARK-2243). To ignore this error, set spark.driver.allowMultipleContexts = true. The currently running SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand

In [ ]:
# Load all datasets
metrics = spark.read.load('../data/dataset_metrics.parquet').toPandas()
score_metrics = spark.read.load('../data/score_metrics.parquet').toPandas().set_index('score')
comments = spark.read.load('../data/sample.parquet')
comments.registerTempTable("comments")
daily_metrics = spark.read.load('../data/daily_metrics.parquet').toPandas().set_index('created')
subreddit_agreement = spark.read.load('../data/subreddit_agreement.parquet').toPandas().set_index('subreddit')
daily_agreement = spark.read.load('../data/daily_agreement.parquet').toPandas().set_index('created')
lda_result_one_w_df = spark.read.load('../data/oneW_oneT_lda_result.parquet').toPandas().set_index('post_id')
lda_result_three_w_df = spark.read.load('../data/threeW_twoT_lda_result.parquet').toPandas().set_index('post_id')
nlp = spark.read.load('../data/subreddit_nlp_full_0.001.parquet')
nlp.registerTempTable("nlp")
nlp_per_community = spark.read.load('../data/nlp_per_community.parquet')
agreement_per_community = spark.read.load('../data/agreement_per_community.parquet').toPandas().set_index('Group')

In [ ]:
spark.sql("""
    SELECT 
        subreddits.Group, 
        COUNT(*) as count, 
        AVG(score) as avg_score,
        STD(score) as std_score,
        STD(score) / AVG(score) as avg_std_score,
        SUM(CASE WHEN score > 1 THEN 1 ELSE 0 END) AS count_pos,
        SUM(CASE WHEN score < 1 THEN 1 ELSE 0 END) AS count_neg,
        (SUM(CASE WHEN score > 1 THEN 1 ELSE 0 END) / COUNT(*)) AS agreement_factor
    FROM comments JOIN subreddits ON LOWER(comments.subreddit) == subreddits.subreddit
    WHERE score > 5 OR score < -3
    GROUP BY subreddits.Group
""").write.mode('overwrite').parquet('agreement_per_community.parquet')

In [ ]:
spark.sql("""
SELECT id, creation_date, body, nltk_negativity, nltk_positivity, nltk_neutrality, nb_bw_matches, nlp.subreddit, subreddits.Group, subreddits.Subgroup
FROM nlp JOIN subreddits 
ON LOWER(nlp.subreddit) == subreddits.subreddit
""").write.mode('overwrite').parquet('nlp_filtered_for_communities.parquet')

In [ ]:
spark.sql("""
SELECT 
    id, 
    creation_date, 
    body, 
    nltk_negativity AS Negativity, 
    nltk_positivity AS Positivity,
    nltk_neutrality AS Neutrality,
    nb_bw_matches AS Vulgarity,
    nlp.subreddit, 
    subreddits.Group, 
    subreddits.Subgroup
FROM nlp JOIN subreddits ON LOWER(nlp.subreddit) == subreddits.subreddit
""").write.mode('overwrite').parquet('nlp_per_community.parquet')